In [163]:
import pandas as pd
import duckdb as db
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 

In [164]:
# Load dataset
data = pd.read_csv("C:/Users/Admin/Downloads/OnlineRetail/OnlineRetail.csv", encoding="ISO-8859-1")

# Standardize InvoiceDate
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], errors='coerce', dayfirst=True)
data = data[data['InvoiceDate'].notnull()]  # Drop invalid dates
data['InvoiceDate'] = data['InvoiceDate'].dt.strftime('%Y-%m-%d')


# Quick check
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-12,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-01-12,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-01-12,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-01-12,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-01-12,3.39,17850.0,United Kingdom


In [165]:
data = data.dropna(subset = ['CustomerID'])

In [168]:
# Connect to DuckDB
con = db.connect()
con.register('data', data)

# ==============================
# 1. Clean Data & Fill Descriptions
# ==============================
data = con.query("""
WITH cleaned_desc AS (
    SELECT 
        StockCode,
        MAX(Description) AS Real_Desc
    FROM data
    WHERE Description IS NOT NULL
    GROUP BY StockCode
)
SELECT 
    d.InvoiceNo,
    d.StockCode,
    COALESCE(d.Description, r.Real_Desc) AS Description,
    d.InvoiceDate,
    CAST(d.Quantity AS INT) AS Quantity,
    CAST(d.UnitPrice AS DOUBLE) AS UnitPrice,
    d.CustomerID,
    d.Country,
    CAST(d.Quantity AS INT) * CAST(d.UnitPrice AS DOUBLE) AS Revenue
FROM data AS d
LEFT JOIN cleaned_desc AS r
    ON d.StockCode = r.StockCode
WHERE Quantity > 0 
  AND UnitPrice > 0
  AND COALESCE(d.Description, r.Real_Desc) IS NOT NULL
""").df()

In [172]:
import os

# ============================
# Define base save path
# ============================
base_path = r"C:/Users/Admin/Downloads/Online Retail/Landing_Page"
os.makedirs(base_path, exist_ok=True)

# ============================
# 1. KPI Summary
# ============================
kpi_summary = con.query("""
SELECT 
    COALESCE(SUM(Revenue), 0) AS Total_Revenue,
    COALESCE(SUM(Quantity), 0) AS Total_Quantity,
    COUNT(DISTINCT InvoiceNo) AS Total_Orders,
    COUNT(DISTINCT CustomerID) AS Customer_Count,
    CASE 
        WHEN COUNT(DISTINCT InvoiceNo) > 0 THEN SUM(Revenue) * 1.0 / COUNT(DISTINCT InvoiceNo)
        ELSE 0
    END AS Average_Order_Value
FROM data
""").df()
kpi_summary.to_csv(os.path.join(base_path, "kpi_summary.csv"), index=False)

# ============================
# 2. Revenue Trend
# ============================
revenue_trend = con.query("""
SELECT 
    STRFTIME(CAST(InvoiceDate AS DATE), '%Y-%m-01') AS Month,
    COALESCE(SUM(Revenue), 0) AS Total_Revenue,
    COALESCE(SUM(Quantity), 0) AS Total_Quantity,
    CASE 
        WHEN COUNT(DISTINCT InvoiceNo) > 0 THEN SUM(Revenue) * 1.0 / COUNT(DISTINCT InvoiceNo)
        ELSE 0
    END AS Avg_Order_Value
FROM data
GROUP BY Month
ORDER BY Month ASC
""").df()

# Convert Month column to datetime in pandas
revenue_trend['Month'] = pd.to_datetime(revenue_trend['Month'])
revenue_trend.to_csv(os.path.join(base_path, "revenue_trend.csv"), index=False)

# ============================
# 3. Top Products
# ============================
top_products = con.query("""
SELECT 
    StockCode, 
    Description,
    COALESCE(SUM(Revenue), 0) AS Product_Revenue,
    COALESCE(SUM(Quantity), 0) AS Product_Quantity
FROM data
GROUP BY StockCode, Description
ORDER BY Product_Revenue DESC
LIMIT 10
""").df()
top_products.to_csv(os.path.join(base_path, "top_products.csv"), index=False)

# ============================
# 4. Revenue by Country
# ============================
country_sales = con.query("""
SELECT 
    Country,
    COALESCE(SUM(Revenue), 0) AS Country_Revenue,
    COUNT(DISTINCT InvoiceNo) AS Orders,
    COUNT(DISTINCT CustomerID) AS Customers
FROM data
GROUP BY Country
ORDER BY Country_Revenue DESC
""").df()
country_sales.to_csv(os.path.join(base_path, "country_sales.csv"), index=False)

print(f"✅ All landing page CSVs saved in: {base_path}")


✅ All landing page CSVs saved in: C:/Users/Admin/Downloads/Online Retail/Landing_Page


In [130]:
con.query("""
   select
   max(InvoiceDate) as most_recent_date
   from data""") 

┌──────────────────┐
│ most_recent_date │
│     varchar      │
├──────────────────┤
│ 2011-12-10       │
└──────────────────┘

In [173]:

# ============================
# Define base save path
# ============================
base_path = r"C:/Users/Admin/Downloads/Online Retail/Segments"
os.makedirs(base_path, exist_ok=True)

# ============================
# Segment KPIs Query
# ============================
segment_kpis_fixed = con.query("""
WITH Rfm_Score AS (
    SELECT 
        CustomerID,
        DATEDIFF('day', MAX(CAST(InvoiceDate AS DATE)), CAST('2011-12-10' AS DATE)) AS recency,
        COUNT(DISTINCT InvoiceNo) AS frequency,
        COALESCE(SUM(CASE WHEN InvoiceNo NOT LIKE 'C%' THEN Revenue END), 0) AS monetary
    FROM data
    GROUP BY CustomerID
),
RankedRFM AS (
    SELECT
        CustomerID,
        recency,
        frequency,
        monetary,
        NTILE(5) OVER (ORDER BY recency ASC) AS R_Score,
        NTILE(5) OVER (ORDER BY frequency DESC) AS F_Score,
        NTILE(5) OVER (ORDER BY monetary DESC) AS M_Score
    FROM Rfm_Score
),
RFM_Final AS (
    SELECT
        CustomerID,
        recency,
        frequency,
        monetary,
        CASE
            WHEN R_Score >= 4 AND F_Score >= 4 AND M_Score >= 4 THEN 'Champions'
            WHEN R_Score >= 3 AND F_Score >= 4 AND M_Score >= 4 THEN 'Loyal'
            WHEN R_Score >= 3 AND F_Score >= 3 AND M_Score >= 3 THEN 'Potential Loyalist'
            WHEN R_Score <= 2 AND F_Score <= 2 AND M_Score <= 2 THEN 'Hibernating'
            ELSE 'At Risk'
        END AS Customer_Segment
    FROM RankedRFM
),
SegmentTotals AS (
    SELECT
        r.Customer_Segment,
        COUNT(DISTINCT r.CustomerID) AS TotalCustomers,
        COUNT(DISTINCT d.InvoiceNo) AS TotalOrders,
        SUM(CASE WHEN d.InvoiceNo NOT LIKE 'C%' THEN d.Revenue END) AS TotalRevenue,
        AVG(SUM(CASE WHEN d.InvoiceNo NOT LIKE 'C%' THEN d.Revenue END)) OVER(PARTITION BY r.Customer_Segment) AS AvgCLV
    FROM data d
    JOIN RFM_Final r ON d.CustomerID = r.CustomerID
    GROUP BY r.Customer_Segment
),
ReturningCustomers AS (
    SELECT
        r.Customer_Segment,
        COUNT(DISTINCT r.CustomerID) AS ReturningCustomers
    FROM data d
    JOIN RFM_Final r ON d.CustomerID = r.CustomerID
    GROUP BY r.Customer_Segment
    HAVING COUNT(DISTINCT d.InvoiceNo) > 1
)
SELECT 
    s.Customer_Segment,
    s.TotalCustomers,
    s.TotalOrders,
    s.TotalRevenue,
    s.TotalOrders * 1.0 / s.TotalCustomers AS AvgPurchaseFrequency,
    COALESCE(r.ReturningCustomers * 100.0 / s.TotalCustomers, 0) AS RetentionRate,
    100 - COALESCE(r.ReturningCustomers * 100.0 / s.TotalCustomers, 0) AS ChurnPercentage,
    s.AvgCLV
FROM SegmentTotals s
LEFT JOIN ReturningCustomers r ON s.Customer_Segment = r.Customer_Segment
ORDER BY TotalRevenue DESC;
""").df()

# ============================
# Save to CSV
# ============================
segment_kpis_fixed.to_csv(os.path.join(base_path, "segment_kpis.csv"), index=False)

print(f"✅ Segment KPIs saved to: {os.path.join(base_path, 'segment_kpis.csv')}")

✅ Segment KPIs saved to: C:/Users/Admin/Downloads/Online Retail/Segments\segment_kpis.csv


In [174]:
# ============================
# Define base save path
# ============================
base_path = r"C:/Users/Admin/Downloads/Online Retail/Product_Analytics"
os.makedirs(base_path, exist_ok=True)

# ============================
# Product Stats Query
# ============================


product_stats = db.query("""
WITH ProductRevenue AS (
    -- Monthly revenue per product
    SELECT
        StockCode,
        Description,
        DATE_TRUNC('month', CAST(InvoiceDate AS DATE)) AS Month,
        SUM(Revenue) AS Revenue
    FROM data
    GROUP BY StockCode, Description, Month
),

RevenueWithMoM AS (
    -- Add previous month revenue for MoM calculation
    SELECT
        StockCode,
        Description,
        Month,
        Revenue,
        LAG(Revenue) OVER (PARTITION BY StockCode ORDER BY Month) AS PrevMonthRevenue
    FROM ProductRevenue
),

RevenueWithMoMYOY AS (
    -- Add same month last year revenue for YoY calculation
    SELECT
        StockCode,
        Description,
        Month,
        Revenue,
        PrevMonthRevenue,
        LAG(Revenue, 12) OVER (PARTITION BY StockCode ORDER BY Month) AS PrevYearRevenue
    FROM RevenueWithMoM
),

-- Calculate total revenue per month for contribution %
MonthlyTotals AS (
    SELECT
        Month,
        SUM(Revenue) AS TotalRevenue
    FROM ProductRevenue
    GROUP BY Month
)

SELECT
    r.StockCode,
    r.Description,
    r.Month,
    r.Revenue,
    COALESCE(r.PrevMonthRevenue, 0) AS PrevMonthRevenue,
    CASE 
        WHEN r.PrevMonthRevenue IS NULL OR r.PrevMonthRevenue = 0 THEN 0
        ELSE (r.Revenue - r.PrevMonthRevenue) * 100.0 / r.PrevMonthRevenue
    END AS MoM_Growth_Percent,
    COALESCE(r.PrevYearRevenue, 0) AS PrevYearRevenue,
    CASE 
        WHEN r.PrevYearRevenue IS NULL OR r.PrevYearRevenue = 0 THEN 0
        ELSE (r.Revenue - r.PrevYearRevenue) * 100.0 / r.PrevYearRevenue
    END AS YoY_Growth_Percent,
    t.TotalRevenue,
    r.Revenue * 100.0 / t.TotalRevenue AS Contribution_Percent
FROM RevenueWithMoMYOY r
JOIN MonthlyTotals t
    ON r.Month = t.Month
ORDER BY r.Month asc, r.Revenue asc;
""").df()


# ============================
# Save to CSV
# ============================
product_stats.to_csv(os.path.join(base_path, "product_stats.csv"), index=False)

print(f"✅ Product stats saved to: {os.path.join(base_path, 'product_stats.csv')}")

✅ Product stats saved to: C:/Users/Admin/Downloads/Online Retail/Product_Analytics\product_stats.csv


In [161]:
product_stats

,StockCode,Description,Month,Revenue,PrevMonthRevenue,MoM_Growth_Percent,PrevYearRevenue,YoY_Growth_Percent,TotalRevenue,Contribution_Percent
0,21917,SET 12 KIDS WHITE CHALK STICKS,2010-01-01,0.42,0.00,0.000000,0.00,0.000000,46376.49,0.000906
1,22135,LADLE LOVE HEART PINK,2010-01-01,0.42,0.00,0.000000,0.00,0.000000,46376.49,0.000906
2,16258A,SWIRLY CIRCULAR RUBBERS IN BAG,2010-01-01,0.42,0.00,0.000000,0.00,0.000000,46376.49,0.000906
3,82580,BATHROOM METAL SIGN,2010-01-01,0.55,0.00,0.000000,0.00,0.000000,46376.49,0.001186
4,82578,KITCHEN METAL SIGN,2010-01-01,0.55,0.00,0.000000,0.00,0.000000,46376.49,0.001186
...,...,...,...,...,...,...,...,...,...,...
37459,47566,PARTY BUNTING,2011-12-01,1716.00,2669.95,-35.729134,93.00,1745.161290,205190.80,0.836295
37460,71477,COLOUR GLASS. STAR T-LIGHT HOLDER,2011-12-01,1761.56,542.76,224.555973,77.35,2177.388494,205190.80,0.858499
37461,POST,POSTAGE,2011-12-01,2541.50,2472.95,2.771993,578.00,339.705882,205190.80,1.238603
37462,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2011-12-01,3999.42,11107.60,-63.993842,2656.80,50.535230,205190.80,1.949122


In [41]:
# Convert InvoiceDate to datetime
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Extract time features
data['Year'] = data['InvoiceDate'].dt.year
data['Month'] = data['InvoiceDate'].dt.month
data['Quarter'] = data['InvoiceDate'].dt.quarter
data['DayOfWeek'] = data['InvoiceDate'].dt.day_name()

data.head()


,InvoiceNo,StockCode,Description,InvoiceDate,Quantity,UnitPrice,CustomerID,Country,IsCancellation,Revenue,Year,Month,Quarter,DayOfWeek
0,545019,22991,GIRAFFE WOODEN RULER,2011-02-25 13:56:00,48,1.95,15093.0,United Kingdom,0,93.60,2011,2,1,Friday
1,545019,22992,REVOLVER WOODEN RULER,2011-02-25 13:56:00,48,1.95,15093.0,United Kingdom,0,93.60,2011,2,1,Friday
2,545019,22636,CHILDS BREAKFAST SET CIRCUS PARADE,2011-02-25 13:56:00,8,7.65,15093.0,United Kingdom,0,61.20,2011,2,1,Friday
3,545019,21507,"ELEPHANT, BIRTHDAY CARD,",2011-02-25 13:56:00,24,0.42,15093.0,United Kingdom,0,10.08,2011,2,1,Friday
4,545019,21506,"FANCY FONT BIRTHDAY CARD,",2011-02-25 13:56:00,24,0.42,15093.0,United Kingdom,0,10.08,2011,2,1,Friday


In [51]:
db.query("Select Country,Sum(Revenue) from data group by Country order by Sum(Revenue) asc")

┌──────────────────────┬────────────────────┐
│       Country        │    sum(Revenue)    │
│       varchar        │       double       │
├──────────────────────┼────────────────────┤
│ Saudi Arabia         │             145.92 │
│ Bahrain              │             754.14 │
│ Czech Republic       │             826.74 │
│ RSA                  │ 1002.3099999999998 │
│ Brazil               │ 1143.6000000000001 │
│ European Community   │ 1300.2500000000002 │
│ Lithuania            │            1661.06 │
│ Lebanon              │ 1693.8800000000003 │
│ United Arab Emirates │ 1902.2800000000002 │
│ Malta                │ 2725.5899999999997 │
│   ·                  │          ·         │
│   ·                  │          ·         │
│   ·                  │          ·         │
│ Sweden               │ 38378.329999999994 │
│ Belgium              │  41196.34000000001 │
│ Switzerland          │  57089.90000000001 │
│ Spain                │ 61577.110000000044 │
│ Australia            │ 138521.31

In [ ]:
# Customer-level aggregation (for RFM/CLV)
customer_df = data.groupby('CustomerID').agg({
    'Revenue': 'sum',
    'InvoiceNo': 'nunique',
    'Quantity': 'sum',
    'InvoiceDate': 'max'
}).reset_index().rename(columns={
    'InvoiceNo': 'NumPurchases',
    'Quantity': 'TotalQuantity',
    'Revenue': 'TotalRevenue',
    'InvoiceDate': 'LastPurchaseDate'
})

customer_df.head()


,CustomerID,TotalRevenue,NumPurchases,TotalQuantity,LastPurchaseDate
0,12346.0,77183.60,1,74215,2011-01-18 10:01:00
1,12347.0,4310.00,7,2458,2011-12-07 15:52:00
2,12348.0,1797.24,4,2341,2011-09-25 13:13:00
3,12349.0,1757.55,1,631,2011-11-21 09:51:00
4,12350.0,334.40,1,197,2011-02-02 16:01:00


In [60]:
# Product-level aggregation (for bestsellers, returns, promos)
product_df = data.groupby(['StockCode', 'Description']).agg({
    'Revenue': 'sum',
    'Quantity': 'sum',
    'InvoiceNo': 'nunique',
    'IsPromo': 'sum',
    'IsCancellation': 'sum'
}).reset_index().rename(columns={
    'InvoiceNo': 'NumTransactions',
    'IsPromo': 'PromoCount',
    'IsCancellation': 'CancellationCount'
})

# Add return rate
product_df['ReturnRate'] = product_df['CancellationCount'] / (product_df['NumTransactions'] + 1e-6)

product_df.head()


KeyError: "Column(s) ['IsPromo'] do not exist"

In [ ]:
# Monthly sales aggregation
monthly_sales = data.groupby(['Year', 'Month']).agg({
    'Revenue': 'sum'
}).reset_index()

plt.figure(figsize=(12,6))
sns.lineplot(x=pd.to_datetime(monthly_sales['Year'].astype(str) + '-' + monthly_sales['Month'].astype(str) + '-01'),
             y='Revenue', data=monthly_sales, marker='o')
plt.title("Monthly Revenue Trend")
plt.xlabel("Date")
plt.ylabel("Revenue")
plt.xticks(rotation=45)
plt.show()
